In [1]:
using CSV 
using DataFrames, Random
using Clustering

# set random seed
Random.seed!(1234)

TaskLocalRNG()

In [2]:
df_MIRLIR = CSV.read("../Results/MIR_LIR_fits2024-09-20-11_43.csv", DataFrame)

Row,ID,Meal Test,Health status,k1,k5,k6,k8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,0.0,0.0102168,0.0459655,2.36046,4.64885
2,1.0,2.0,0.0,0.010703,0.039215,2.62905,4.89013
3,1.0,3.0,0.0,0.0114941,0.0361941,2.43105,4.68702
4,1.0,4.0,0.0,0.0120577,0.0408216,2.21771,3.82294
5,1.0,5.0,0.0,0.0128715,0.0362616,2.31327,3.81754
6,1.0,6.0,0.0,0.0140081,0.0300844,2.41778,4.03432
7,2.0,1.0,0.0,0.0102496,0.0467434,2.24555,4.55563
8,2.0,2.0,0.0,0.0106976,0.0462139,2.25448,4.24349
9,2.0,3.0,0.0,0.0113883,0.0405245,2.28657,4.2954


In [3]:
#df_fits =  CSV.read("../Results/Synthetic_population_fits.csv", DataFrame)

In [4]:
# Rename the columns to be used in unstack
df_k5s = rename(df_MIRLIR,  Symbol("Meal Test") => :parameter, :k6 => :value)
# Get the unique IDs and Health status
df_health_status = unique(df_MIRLIR[:, [:ID, Symbol("Health status")]])

# Unstack the DataFrame
df_unstacked = unstack(df_k5s, :ID,  :parameter, :value)
# Join the "Health status" DataFrame with the unstacked DataFrame
df_unstacked = rightjoin(df_unstacked, df_health_status, on = :ID)
#R = kmeans(k5_data,2)


# Get the data for the clustering
k5_data = df_unstacked[:, 2:end-1]
k5_data = Matrix{Float64}(k5_data)	
# Perform the clustering
R = kmeans(k5_data', 2)
# Find the missing values

a = assignments(R) # get the assignments of points to clusters
c = counts(R) # get the cluster sizes



2-element Vector{Int64}:
 15
  5

In [5]:
using MultivariateStats, Plots

# Perform PCA
M = fit(PCA, k5_data'; maxoutdim=2)

# Project the data onto the principal components
Yte = MultivariateStats.transform(M, k5_data')

# Get the variance explained by each component
ve = round.(M.prinvars / sum(M.prinvars);digits=4)
ve1 = ve[1]
ve2 = ve[2]

# Create a scatter plot of the projected data, colored by cluster assignment
scatter(Yte[1, :], Yte[2, :], color=a, legend=false, xlabel="PC1 ($ve1%)" , ylabel="PC2 ($ve2%)", title="PCA parameter k5")

savefig("k5_clusters.png")


"c:\\Users\\20192809\\Documents\\GitHub\\EDES\\Legacy projects\\k5_clusters.png"

In [6]:
# implement a elbow plot
using Clustering
using Plots

# Create a vector to store the sum of squared errors
sse = Float64[]
# Create a vector to store the number of clusters
k = 1:10
# Loop over the number of clusters
for i in k
    # Perform the clustering
    R = kmeans(k5_data', i)
    # Get the sum of squared errors
    push!(sse, sum(R.dists))
end

# Create a plot of the sum of squared errors
plot(k, sse, xlabel="Number of clusters", ylabel="Sum of squared errors", title="Elbow plot for k5")


ErrorException: type KmeansResult has no field dists